# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [16]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /Users/knishina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/knishina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/knishina/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [19]:
# import libraries
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.base import BaseEstimator, TransformerMixin

In [3]:
def load_data():
    # load data from database
    engine = create_engine('sqlite:///DisasterResponse.db')
    df = pd.read_sql('disaster_response', con=engine)
    
    # split data into X and y.
    X = df["message"]
    y = df.iloc[:, 4:]
    
    # split data into train and test.
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    # obtain column names.
    column_names = list(y.columns)
    
    return X_train, X_test, y_train, y_test, column_names

In [4]:
X_train, X_test, y_train, y_test, column_names = load_data()

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # replace the webpage with a urlplaceholder
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # tokenize text
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # lemmatize, turn into lowercase, strip spaces
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
def model_pipeline():
    pipeline = Pipeline([
    ("vect", CountVectorizer(tokenizer=tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
model = model_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
results = model.predict(X_test)

In [9]:
# sanity check on the expected output labels.
foo = results[0]
foobar = np.where(foo == 1)[0]
bar = [column_names[x] for x in foobar]
bar

['related', 'aid_related']

In [10]:
# overall accuracy of the model.
overall_accuracy = (results == y_test).mean().mean()
print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

Average overall accuracy 94.26% 



In [11]:
# scores for each category using classification_report.
y_pred_pd = pd.DataFrame(results, columns = column_names)
for column in column_names:
    print(f"FEATURE: {column}")
    print(classification_report(y_test[column],y_pred_pd[column]))
    print("------------------------------------------------------\n")

FEATURE: related
             precision    recall  f1-score   support

          0       0.62      0.36      0.46      1531
          1       0.82      0.93      0.87      4970
          2       0.71      0.23      0.34        44

avg / total       0.77      0.79      0.77      6545

------------------------------------------------------

FEATURE: request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5432
          1       0.80      0.36      0.49      1113

avg / total       0.87      0.88      0.86      6545

------------------------------------------------------

FEATURE: offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6510
          1       0.00      0.00      0.00        35

avg / total       0.99      0.99      0.99      6545

------------------------------------------------------

FEATURE: aid_related
             precision    recall  f1-score   support

          

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
# add custom estimator.
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [13]:
# improve the pipeline!
def new_model_pipeline():
    pipeline = Pipeline([
        ("features",  FeatureUnion([
            ("nlp_pipeline", Pipeline([
                ("vect", CountVectorizer(tokenizer=tokenize)),
                ("tfidf", TfidfTransformer())
            ])),
            ("start_verb", StartingVerbExtractor())
        ])),
        ("clf", MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    return pipeline

In [18]:
# train the new model
model = new_model_pipeline()
model.fit(X_train, y_train)

# test the new model.
results = model.predict(X_test)

# overall accuracy of the model.
overall_accuracy = (results == y_test).mean().mean()
print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

# scores for each category using classification_report.
y_pred_pd = pd.DataFrame(results, columns = column_names)
for column in column_names:
    print(f"FEATURE: {column}")
    print(classification_report(y_test[column],y_pred_pd[column]))
    print("------------------------------------------------------\n")

Average overall accuracy 94.14% 

FEATURE: related
             precision    recall  f1-score   support

          0       0.62      0.33      0.43      1527
          1       0.81      0.94      0.87      4966
          2       0.60      0.06      0.11        52

avg / total       0.77      0.79      0.76      6545

------------------------------------------------------

FEATURE: request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5423
          1       0.81      0.36      0.50      1122

avg / total       0.87      0.88      0.86      6545

------------------------------------------------------

FEATURE: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6514
          1       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6545

------------------------------------------------------

FEATURE: aid_related
             precision    reca

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.